In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image

In [2]:
class SkinDiseaseDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.labels_frame.iloc[idx, 0])
        image = Image.open(img_name+'.jpg')
        labels = self.labels_frame.iloc[idx, 1:].values
        #labels = torch.argmax(torch.tensor(labels.astype('float')))
        #labels = torch.argmax(torch.tensor(labels.astype('float'))).unsqueeze(0)
        labels = torch.argmax(torch.tensor(labels.astype('float'))).unsqueeze(0).squeeze()


        
        if self.transform:
            image = self.transform(image)

        return image, labels



class SkinDiseaseClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SkinDiseaseClassifier, self).__init__()
        #base =models.resnet152(pretrained=True)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        #self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        #self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        #x = self.pool(self.relu(self.bn4(self.conv4(x))))
        #print("X.shape")
        #print(x.shape)
        #x = x.view(-1, 256 * 28 * 28)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_labels_df = pd.read_csv('/workspace/ConditionalGAN/Dataset/Train/Train_labels.csv')
test_labels_df = pd.read_csv('/workspace/ConditionalGAN/Dataset/Test/Test_Labels.csv')

In [7]:
label_encoder = LabelEncoder()
train_labels_df['label'] = label_encoder.fit_transform(train_labels_df[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values.argmax(axis=1))
test_labels_df['label'] = label_encoder.transform(test_labels_df[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values.argmax(axis=1))

In [8]:
# Start
train_dataset = SkinDiseaseDataset(csv_file='/workspace/ConditionalGAN/Dataset/Train/Train_labels.csv', root_dir='/workspace/ConditionalGAN/Dataset/Train/Train_data', transform=transform)
test_dataset = SkinDiseaseDataset(csv_file='/workspace/ConditionalGAN/Dataset/Test/Test_Labels.csv', root_dir='/workspace/ConditionalGAN/Dataset/Test/Test_data', transform=transform)

#end

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True ,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False , drop_last=True)

In [15]:
lr = 0.001
num_epochs = 15
num_classes = 7

classifier = SkinDiseaseClassifier(num_classes)


optimizer = optim.Adam(classifier.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier.to(device)
criterion.to(device)

CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
    classifier.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)


        optimizer.zero_grad()


        outputs = classifier(images)
        
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track the accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Evaluate the classifier
classifier.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save the trained classifier
torch.save(classifier.state_dict(), 'skin_disease_classifier.pth')

In [16]:
import torchvision.models as models

# Load the saved state dictionary
checkpoint = torch.load('skin_disease_classifier.pth')

# Load the state dictionary into the model
classifier.load_state_dict(checkpoint)

SkinDiseaseClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
)

In [23]:
test_dataset = SkinDiseaseDataset(csv_file='/workspace/ConditionalGAN/Dataset/Test/Test_Labels.csv', root_dir='/workspace/ConditionalGAN/images', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False , drop_last=True)

In [25]:
# Evaluate the classifier
classifier.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.4f}')